In [1]:
## MLFLOW_TRACKING_URI='https://qualiextra-project-6b29e931dd67.herokuapp.com/'

In [2]:
import pandas as pd
import datetime


In [3]:
data = pd.read_csv("./Missions.csv", sep=';')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4771 entries, 0 to 4770
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Propriété      4764 non-null   object 
 1   hôtel          4760 non-null   object 
 2   extra          4761 non-null   object 
 3   date           4771 non-null   object 
 4   statuts        617 non-null    object 
 5   tarif urgence  225 non-null    float64
 6   tarif horaire  4760 non-null   float64
 7   nbre d'heures  4771 non-null   object 
 8   total HT       4771 non-null   int64  
 9   facture        3798 non-null   object 
 10  règlement      3743 non-null   object 
 11  montant HT     4771 non-null   int64  
 12  Texte          7 non-null      object 
dtypes: float64(2), int64(2), object(9)
memory usage: 484.7+ KB


In [5]:
mask = ((data["hôtel"].notna()))
data = data[mask]
data.shape

(4760, 13)

In [6]:
data["date_debut"] = data.apply(lambda x: x["date"].split(" →")[0],axis=1)
data["date_debut"] = data["date_debut"].apply(lambda x: x.replace(" (UTC+3)", ""))
data["date_debut"] = data["date_debut"].apply(lambda x: x.replace(" (UTC)", ""))

In [7]:
data["date_debut"] = pd.to_datetime(data["date_debut"], format="%d/%m/%Y %H:%M")

In [8]:
data.dtypes

Propriété                object
hôtel                    object
extra                    object
date                     object
statuts                  object
tarif urgence           float64
tarif horaire           float64
nbre d'heures            object
total HT                  int64
facture                  object
règlement                object
montant HT                int64
Texte                    object
date_debut       datetime64[ns]
dtype: object

In [9]:
data["time_delta"] = data["nbre d'heures"].apply(lambda x: pd.to_timedelta(x))
data["date_fin"] = data.apply(lambda x: x["date_debut"] + x["time_delta"], axis=1)
data.head()

,Propriété,hôtel,extra,date,statuts,tarif urgence,tarif horaire,nbre d'heures,total HT,facture,règlement,montant HT,Texte,date_debut,time_delta,date_fin
0,Crystal,Crystal (https://www.notion.so/Crystal-1837fcb...,Grégory Tshimini (https://www.notion.so/Gr-gor...,07/01/2024 8:00 → 20:00,NaN,NaN,21.0,12hr 0min,252,NaN,NaN,180,NaN,2024-01-07 08:00:00,0 days 12:00:00,2024-01-07 20:00:00
1,Milie rose,Milie Rose (https://www.notion.so/Milie-Rose-0...,Mamadou Koma (https://www.notion.so/Mamadou-Ko...,06/01/2024 22:30 → 07/01/2024 8:00,NaN,NaN,20.0,9hr 30min,180,NaN,NaN,135,NaN,2024-01-06 22:30:00,0 days 09:30:00,2024-01-07 08:00:00
2,Touraine,Touraine (https://www.notion.so/Touraine-df4e7...,Mohamed Belbachir (https://www.notion.so/Moham...,06/01/2024 19:30 → 07/01/2024 7:30,NaN,NaN,20.0,12hr 0min,240,NaN,NaN,180,NaN,2024-01-06 19:30:00,0 days 12:00:00,2024-01-07 07:30:00
3,Crystal,Crystal (https://www.notion.so/Crystal-1837fcb...,Grégory Tshimini (https://www.notion.so/Gr-gor...,06/01/2024 8:00 → 20:00,NaN,NaN,21.0,12hr 0min,252,NaN,NaN,180,NaN,2024-01-06 08:00:00,0 days 12:00:00,2024-01-06 20:00:00
4,Milie rose,Milie Rose (https://www.notion.so/Milie-Rose-0...,Mamadou Koma (https://www.notion.so/Mamadou-Ko...,05/01/2024 22:30 → 06/01/2024 8:00,NaN,NaN,20.0,9hr 30min,180,NaN,NaN,135,NaN,2024-01-05 22:30:00,0 days 09:30:00,2024-01-06 08:00:00


In [10]:
data["date_debut"].dt.year.value_counts()

date_debut
2023    2827
2022    1841
2021      82
2024      10
Name: count, dtype: int64

In [13]:
import plotly.express as px

px.histogram(data["date_debut"].dt.year.astype(str))